In [26]:
import pandas as pd
from statistics import mean, median
import sys
import pprint
import pickle

In [27]:
pp = pprint.PrettyPrinter(indent=4)

In [28]:
batch_sizes = [2, 4, 8, 16]
batching_alg = ['BATCH4', 'BATCHBISECT', 'BATCHSTOP4']
projects = []
num = 1

In [29]:
def get_results(results):
    l = []
    
    for r in results:
        if r == 'passed':
            l.append(1)
        else:
            l.append(0)
            
    return l

In [30]:
def str_to_list(s):
    if s == '[]':
        return [0]
    l = s[1:-1].split(', ')
    l = [int(x) for x in l]
    return l

In [31]:
def get_empty_library():
    global projects
    ssr_delay_lib = {}
    
    for p in projects:
        ssr_delay_lib[p] = {}
        for v in range(1, 11):
            ssr_delay_lib[p][v] = {'BATCHSTOP4':{4:{}, 8:{}, 16:{}}, 'BATCH4':{4:{}}, 'BATCHBISECT':{1:{}, 2:{}, 4:{}, 8:{}, 16:{}}}
    for p in ssr_delay_lib:
        for ver in ssr_delay_lib[p]:
            for alg in ssr_delay_lib[p][ver]:
                for b in ssr_delay_lib[p][ver][alg]:
                    for i in range(2,21):
                        ssr_delay_lib[p][ver][alg][b][i] = {}
    
    return ssr_delay_lib
        

In [32]:
def output_values(Y_data):
    Y_t = []
    for e in Y_data:
        if e == 'passed':
            Y_t.append(1)
        else:
            Y_t.append(0) 
    return Y_t

In [33]:
def get_first_failures(df):
    
    results = df['tr_status'].tolist()
    length = len(results)
    verdict = ['keep']
    prev = results[0]
    
    for i in range(1, length):
        if results[i] == 0:
            if prev == 0:
                verdict.append('discard')
                #print(i+1)
            else:
                verdict.append('keep')
        else:
            verdict.append('keep')
        prev = results[i]
    
    df['verdict'] = verdict
    df = df[ df['verdict'] == 'keep' ]
    df.drop('verdict', inplace=True, axis=1)
    return df

In [34]:
def get_delay_lib(ssr_results):
    
    ssr_delay_lib = get_empty_library()
    
    for x in range(len(ssr_results)):
        row = ssr_results.iloc[x]
        ci = str_to_list(row['ci'])
        b = row['batch_size'] - 1
        project = row['project']

        ver = row['version']
        filename = '../../data/project_data_pickles/' + project + '.csv_' + str(ver) + '_indexes.pkl'
        with open(filename, 'rb') as save_file:
            train_build_ids = pickle.load(save_file)
            test_build_ids = pickle.load(save_file)

        test_file = pd.read_csv('../../data/full_data/' + project + '.csv') 
        test_file['tr_status'] = output_values(test_file['tr_status'])
        test_file = get_first_failures(test_file)

        X_test = test_file [ test_file['tr_build_id'].isin(test_build_ids)]
        y_test = X_test['tr_status'].tolist()

        i = 0        
        delay_list = []
        missed = []
        batch_list = []
        sbs_list = []

        while i < len(ci):

            sbs_delay = 0
            batch_delay = 0

            if ci[i] == 0:
                while len(missed) > 0:
                    ind = missed.pop()
                    sbs_delay += i - ind
                    sbs_list.append(i - ind)
                batch_delay += b
                batch_list.append(b)
                b -= 1
                if b == -1:
                    b = row['batch_size'] - 1

            if ci[i] == 1:
                if y_test[i] == 0:
                    missed.append(i)

            delay_list.append(sbs_delay + batch_delay)
            #print(y_test[i], ci[i], sbs_delay, batch_delay, sbs_delay + batch_delay)

            i += 1
        while len(missed) > 0:
                delay_list.append(i - missed.pop())

        ssr_delay_lib[row['project']][row['version']][row['algorithm']][row['batch_size']][row['confidence']]['delays'] = delay_list
    
    return ssr_delay_lib
        

In [35]:
def get_lines(ssr_results, ssr_delay_lib):
    lines = []
    for x in range(len(ssr_results)):
        row = ssr_results.iloc[x]
        d = ssr_delay_lib[row['project']][row['version']][row['algorithm']][row['batch_size']][row['confidence']]['delays']
        m = median(d)
        l = list(filter(lambda num: num != 0, d))
        if l == []:
            l = [0]
        lines.append([row['version'], row['project'], row['algorithm'], row['batch_size'], row['confidence'], row['project_reqd_builds'], row['testall_size'], d, m, median(l)])
    return lines


In [36]:
def start_result_collection(ssr_result_file):
    
    global projects
    global num
    
    #ssr_result_file = '../version_results/cvh_models.csv'
    ssr_results = pd.read_csv(ssr_result_file)
    
    
    projects = set(ssr_results['project'].tolist())
    
    ssr_delay_lib = get_delay_lib(ssr_results)
    lines = get_lines(ssr_results, ssr_delay_lib)
    
    num += 1
    
#     print('This file will generate results for delay list and median delay (from ci).')
#     filename = input('What would you name the result file? (abcd.csv) \n') 
    
    df = pd.DataFrame(lines, columns=['version', 'project', 'algorithm', 'batch_size', 'confidence', 'builds_reqd', 'testall_size', 'delay_list', 'zero_median', 'wo_zero_median'])
#     df.to_csv(filename)
    
    return df

In [37]:
filenames = ['cvh_models.csv', 'lfs.csv', 'results1.csv', 'results2.csv', 'gradle_models.csv', 'rubinius_models.csv']
file_root = '../version_results/'
dfs = []

for file in filenames:
    dfs.append(start_result_collection(file_root+file))

#combining dfs
final_df = pd.DataFrame()
final_df = dfs[0]

for i in range(1, len(dfs)):
    final_df = final_df.append(dfs[i])

final_df.to_csv('final_delay_boxplots.csv')

/opt/homebrew/Caskroom/miniforge/base/envs/new_env/lib/python3.9/site-packages/pandas/core/frame.py:4906: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


# Combining Versions

In [38]:
def separate_versions(results):
    version_dfs = []
    
    for i in range(1,11):
        ver = results[ results['version']==i]
        version_dfs.append(ver)
    return version_dfs

In [48]:
def combine_versions(results):
    
    projects = set(results['project'].tolist())
    all_versions = pd.DataFrame()
        
    for p in projects:
        
        p_data = results[ results['project']==p]
        pframe = pd.DataFrame()
        
        versions = separate_versions(p_data)
        
        for start in range(0,10):
            if len(versions[start]) > 0:
                pframe = versions[start]
                break
        
        for x in range(len(pframe)):
            row = pframe.iloc[x]
            
            alg = row['algorithm']
            b = row['batch_size']
            conf = row['confidence']
            
            final_delay_list = row['delay_list']
            
            for i in range(start+1,10):
                next_ver = versions[i]
                new_df = next_ver[ (next_ver['algorithm']==alg) & (next_ver['batch_size']==b) & (next_ver['confidence']==conf)]
                
                if len(new_df) > 0:
                    new_row = new_df.iloc[0]
                else:
                    continue
                
                total_size = row['testall_size'] + new_row['testall_size']
                total_reqd_builds = (row['builds_reqd']*row['testall_size']) + (new_row['builds_reqd']*new_row['testall_size'])
                final_delay_list.extend(new_row['delay_list'])
                
                row['builds_reqd'] = total_reqd_builds/total_size                
                row['delay_list'] = final_delay_list
                row['testall_size'] = total_size
                
            pframe.iloc[x] = row
        all_versions = all_versions.append(pframe)
    
    return all_versions

In [54]:
delay_df = combine_versions(final_df)


In [56]:
delay_df.drop('delay_list', inplace=True, axis=1)

In [57]:
delay_df.to_csv('combined_delay.csv')